## Limpiar datos iteraciones

In [1]:
import csv, numpy, pandas, os

In [19]:
def cambiar_nombre(old_name, tipo):
    Months = {"Jan": "01", "Feb": "02", "May": "05"}
    i = 0
    month = old_name[4:7]
    month_num = Months[month]
    day = old_name[8:10]  
        
    if len(old_name) == 29:
        i = 1
        day = "0" + old_name[8:9]
        
    time = old_name[11-i:19-i]
    time_2 = time[0:2] + time[3:5] + time [6:8]
    year = old_name[20-i:24-i]
    cmv = old_name[25-i:]

    new_name = year + "-" + month_num + "-" + day + "-" +\
                time_2 + "-" + tipo + "-" + cmv
    
    return(new_name)

In [135]:
iteraciones = []
with open('iteracionesMMporLV.csv') as csvfile:
    datos = csv.reader(csvfile)
    for row in datos:
        iteraciones.append(row)

In [136]:
iteraciones = numpy.array(iteraciones)

In [137]:
iteraciones[1][0] = 'iteraciones eq'
iteraciones[2][0] = 'actual eq'
iteraciones[3][0] = 'nombre eq'

iteraciones[7][0] = 'iteraciones os'
iteraciones[8][0] = 'actual os'
iteraciones[9][0] = 'nombre os'

In [138]:
# Borrar filas, convertir a columnas
mask = numpy.ones(len(iteraciones), dtype=bool)
mask[[0,4,5,6]] = False
iteraciones = iteraciones[mask,...]
iteraciones = iteraciones.transpose()

In [139]:
# Separar comunidades de equilibrio y oscilantes
iteraciones_eq = iteraciones[:, :3]
iteraciones_os = iteraciones[:, 3:]

In [140]:
# Quitar filas vacías
iteraciones_eq = iteraciones_eq[iteraciones_eq[:, 2] != '']
iteraciones_os = iteraciones_os[iteraciones_os[:, 2] != '']

In [141]:
for i, old_name in enumerate(iteraciones_os[1:, 2]):
    iteraciones_os[i+1, 2] = cambiar_nombre(old_name[:30], 'os')[:-6]

In [142]:
for i, old_name in enumerate(iteraciones_eq[1:, 2]):
    iteraciones_eq[i+1, 2] = cambiar_nombre(old_name[:30], 'eq')[:-6]

In [143]:
# Juntar eq y os de nuevo, borrar títulos de col, acomodar
iteraciones = numpy.concatenate((iteraciones_eq, iteraciones_os))
mask = numpy.ones(len(iteraciones), dtype=bool)
mask[[0,97]] = False
iteraciones = iteraciones[mask,...]
iteraciones = iteraciones[numpy.argsort(iteraciones[:, 2]), :]

In [188]:
files = os.listdir("./comunidades")
files.sort()
files = list(filter(lambda x: x[-5] == "m", files))

files = numpy.array(files)
files = numpy.append(files, numpy.zeros((201)))
files = numpy.reshape(files, (201, 2), order='F')

In [189]:
for n, file in enumerate(files):
    try:
        i = iteraciones[iteraciones[:, 2] == file[0][:20]][0, 0]    
        files[n, 1] = int(i[-1])
    except(IndexError): #si no está
        pass

In [190]:
for file in files:
    file[0] = file[0][:20]
    try:
        file[1] = int(file[1])
    except(ValueError): # si no hay datos ie 0.0
        file[1] = 0

In [191]:
filestest = numpy.array2string(files)

In [193]:
numpy.savetxt('iteraciones.txt', files, fmt=['%.20s', '%1s'])

In [220]:
iteracionesf = []
with open('iteraciones.txt', 'r') as file:
    rows = file.readlines()
    for row in rows:
        iteracionesf.append([row[:20], int(row[-2])])